In [19]:
%matplotlib inline
%load_ext ipycache

import pandas as pd
import numpy as np
import scipy
import sklearn as sk
import xgboost as xgb

import matplotlib.pyplot as plt

/Users/evgeny/Library/Python/2.7/lib/python/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
/Users/evgeny/Library/Python/2.7/lib/python/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")
/Users/evgeny/Library/Python/2.7/lib/python/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Загрузка данных

In [31]:
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")
train.head()

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price_doc
0,1,2011-08-20,43,27.0,4.0,NaN,NaN,NaN,NaN,NaN,...,9,4,0,13,22,1,0,52,4,5850000
1,2,2011-08-23,34,19.0,3.0,NaN,NaN,NaN,NaN,NaN,...,15,3,0,15,29,1,10,66,14,6000000
2,3,2011-08-27,43,29.0,2.0,NaN,NaN,NaN,NaN,NaN,...,10,3,0,11,27,0,4,67,10,5700000
3,4,2011-09-01,89,50.0,9.0,NaN,NaN,NaN,NaN,NaN,...,11,2,1,4,4,0,0,26,3,13100000
4,5,2011-09-05,77,77.0,4.0,NaN,NaN,NaN,NaN,NaN,...,319,108,17,135,236,2,91,195,14,16331452


# Препроцессинг фич

In [39]:
train = train.drop(["id", "timestamp", "sub_area"], axis=1)

In [63]:
train.select_dtypes(["object"]).head()

,product_type,ecology
0,Investment,good
1,Investment,excellent
2,Investment,poor
3,Investment,good
4,Investment,excellent


In [73]:
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
onehot_feat = ["product_type", "ecology"]

# onehot_feat = map(list(train.columns).index, onehot_feat)
pre_steps = [
#     OneHotEncoder(categorical_features=onehot_feat)
]
train = train.drop(onehot_feat, axis=1)

In [74]:
bool_feats = [
"thermal_power_plant_raion",
"incineration_raion",
"oil_chemistry_raion",
"radiation_raion",
"railroad_terminal_raion",
"big_market_raion",
"nuclear_reactor_raion",
"detention_facility_raion",
"water_1line",
"big_road1_1line",
"railroad_1line",
]
for bf in bool_feats:
    train[bf] = map(lambda x: x == "yes", train[bf].values)

# Обучение моделей

In [81]:
from sklearn.model_selection import train_test_split
X = train.drop(["price_doc"], axis=1)
y = train[["price_doc"]]

train_X, test_X, train_y, test_y = train_test_split(X.values, y.values, test_size=0.1)

In [82]:
%%cache -f skgbr.pkl model
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import GradientBoostingRegressor

# gbr = GradientBoostingRegressor()
gbr = xgb.XGBRegressor(max_depth=3, n_estimators=200, learning_rate=0.05)

model = make_pipeline(*(pre_steps + [gbr]))
model.fit(train_X, train_y)

Pipeline(steps=[('xgbregressor', XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.05, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=200, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1))])

[Saved variables model to file '/Users/evgeny/PycharmProjects/kaggle/sber/skgbr.pkl'.]


# Валидация

In [83]:
import math

#A function to calculate Root Mean Squared Logarithmic Error (RMSLE)
def rmsle(y, y_pred):
    assert len(y) == len(y_pred)
    terms_to_sum = [(math.log(y_pred[i] + 1) - math.log(y[i] + 1)) ** 2.0 for i,pred in enumerate(y_pred)]
    return (sum(terms_to_sum) * (1.0/len(y))) ** 0.5

In [84]:
rmsle(test_y, model.predict(test_X))

0.46136930415472444